# X (Twitter) Feed Monitor POC
This notebook is a proof of concept to monitor new posts from multiple X/Twitter accounts using Python and Playwright. It avoids the need for Twitter's API by scraping the public timeline using a headless browser.

**Step 1: Install Dependencies and Set Up**


In [1]:
# Install dependencies (only run once)
!pip install playwright nest_asyncio IPython
!playwright install


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## Step 2: Load an X Profile and Get the Latest Tweet
We now use Playwright to open a headless browser, go to an X profile, and extract the first tweet.

In [ ]:
import nest_asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()  # allows async code to run in jupyter notebooks

async def get_first_tweet(username="nasa"):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # For debug, try headless=True after it's stable
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
            viewport={"width": 1280, "height": 800}
        )
        page = await context.new_page()

        print(f"Visiting profile: https://x.com/{username}")
        await page.goto(f"https://x.com/{username}", timeout=60000)

        # Optional: Wait until network activity is idle
        await page.wait_for_load_state("networkidle")

        # Find first tweet using locator instead of selector
        first_tweet = page.locator("article").first

        # Wait until it becomes visible
        await first_tweet.wait_for(timeout=20000)
        tweet_text = await first_tweet.inner_text()

        await browser.close()
        return tweet_text

# Run it:
# await get_first_tweet("nasa")



Visiting profile: https://x.com/nasa


"NASA\n@NASA\n·\n2h\nWe won an Emmy!\n\nOur live broadcast coverage of the 2024 total solar eclipse received this year's Emmy Award for Outstanding Live News Special. Get the details: https://go.nasa.gov/4k9jIpa\nThe media could not be played.\nReload\n101\n206\n1.4K\n185K"

## Step 3: Monitor One User for New Tweets

- Repeats the check every 30 seconds
- Compares the newest tweet with the last one
- Triggers a sound or printed alert if there's a change

In [4]:
import time
from IPython.display import Audio, display

async def monitor_user(username="nasa", check_interval=30):
    last_tweet = None  # stores the previous tweet content

    while True:
        print(f"Checking @{username}...")
        try:
            tweet = await get_first_tweet(username)  # reuse the function from earlier
            if last_tweet is None:
                print("First check. Storing initial tweet.")
                last_tweet = tweet
            elif tweet != last_tweet:
                print(f"🔔 New tweet from @{username}!\n")
                display(Audio(data=b'\x00' * 1000, autoplay=True))  # placeholder sound
                last_tweet = tweet
            else:
                print("No new tweet.\n")
        except Exception as e:
            print(f"⚠️ Error checking @{username}: {e}")
        
        time.sleep(check_interval)  # wait N seconds before checking again


In [5]:
# Run it:
# await monitor_user("nasa", check_interval=30)

## Step 4: Monitor Random Subset of X Profiles
We'll use a predefined list of official CDMX accounts and randomly pick 5 on each cycle. This keeps the browser usage low and avoids detection while still getting broad coverage.

In [14]:
import random

# Extracted handles (no @, no query params)
user_pool = [
    "CDMXConsejeria", "ContraloriaCDMX", "FiscaliaCDMX", "ClaraBrugadaM", "Finanzas_CDMX",
    "SEBIEN_cdmx", "CulturaCiudadMx", "SedecoCDMX", "Vivienda_CDMX", "SECTEI_CDMX",
    "sgirpc_cdmx", "GobCDMX", "semujerescdmx", "SEDEMA_CDMX", "LaSEMOVI",
    "SOBSECDMX", "metropoliscdmx", "sepicdmx", "SSaludCdMx", "SSC_CDMX",
    "TrabajoCDMX", "turismocdmx", "C5_CDMX", "MetrobusCDMX", "Bomberos_CDMX",
    "SEGIAGUA", "UCS_GCDMX", "LaAgenciaCDMX", "DGRCivilCDMX", "DiversidadCDMX",
    "locatel_mx", "SCPPyBG", "SAPCI_CDMX", "icat_cdmx", "CedaGeneral",
    "PDI_FGJCDMX", "CFilmaCDMX", "MetroCDMX", "STECDMX", "micablebuscdmx",
    "RTP_CiudadDeMex", "InjuveCDMX"
]


## Step 5: Monitor 5 Random Users and Alert on New Tweets
We now randomly pick 5 users from the pool each time, check their most recent tweets, and alert if any have changed since last seen.

In [ ]:
import time
from IPython.display import Audio, display

# Dictionary to keep track of last tweets per user
last_tweets = {}

async def monitor_random_users(user_pool, check_interval=60, sample_size=5):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36",
            viewport={"width": 1280, "height": 800}
        )

        while True:
            sample_users = random.sample(user_pool, sample_size)
            print(f"\n🔄 Checking {sample_size} random accounts...")

            for username in sample_users:
                page = await context.new_page()
                try:
                    print(f"Visiting @{username}...")
                    await page.goto(f"https://x.com/{username}", timeout=60000)
                    await page.wait_for_load_state("networkidle")
                    tweet_locator = page.locator("article").first
                    await tweet_locator.wait_for(timeout=15000)
                    tweet_text = await tweet_locator.inner_text()

                    if username in last_tweets:
                        if tweet_text != last_tweets[username]:
                            print(f"🔔 New tweet from @{username}!")
                            display(Audio(data=b'\x00' * 1000, autoplay=True))  # Silent beep
                            last_tweets[username] = tweet_text
                        else:
                            print(f"No new tweet for @{username}.")
                    else:
                        print(f"First time seeing @{username}, storing tweet.")
                        last_tweets[username] = tweet_text

                except Exception as e:
                    print(f"⚠️ Error checking @{username}: {e}")
                finally:
                    await page.close()

            print(f"✅ Cycle complete. Waiting {check_interval} seconds...\n")
            time.sleep(check_interval)


In [16]:
# Run it:
await monitor_random_users(user_pool, check_interval=15, sample_size=5)


🔄 Checking 5 random accounts...
Visiting @icat_cdmx...
First time seeing @icat_cdmx, storing tweet.
Visiting @DiversidadCDMX...
First time seeing @DiversidadCDMX, storing tweet.
Visiting @SCPPyBG...
First time seeing @SCPPyBG, storing tweet.
Visiting @CDMXConsejeria...
First time seeing @CDMXConsejeria, storing tweet.
Visiting @ClaraBrugadaM...
First time seeing @ClaraBrugadaM, storing tweet.
✅ Cycle complete. Waiting 15 seconds...



NameError: name 'time' is not defined